In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Bidirectional, LSTM, Reshape, RepeatVector, TimeDistributed
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import tensorflow_gan as tfgan
import tensorflow as tf
tf.compat.v1.enable_eager_execution() 

import matplotlib.pyplot as plt

import sys

import numpy as np
import pandas as pd

import os

from PIL import Image

# Load Data
Loading from preprocessed numpy array

In [45]:
def load_data():
    x_train = np.array(pd.read_csv("./data_stock/SP500_average.csv",).drop([0]).drop(columns=['Date']))
    return x_train
            

def split_time_series(t, arr) -> (np.array, np.array):
    a = []
    b = []
    for i in range(len(arr)-t):
        a.append(arr[i: i+t])
        b.append(arr[i+t])
    return (np.array(a), np.array(b))
    

def merge_time_series(arr1, arr2) -> tf.float64 :
    t1 = tf.cast(arr1, 'float64') if tf.is_tensor(arr1) else tf.convert_to_tensor(arr1, dtype='float64')
    t2 = tf.cast(arr2, 'float64') if tf.is_tensor(arr2) else tf.convert_to_tensor(arr2, dtype='float64')
    return tf.concat([t1, tf.expand_dims(t2, 1) ], axis=1)

def standardize_time_series(arr1):
    scaler = StandardScaler()
    scaler.fit(arr1)
    return (scaler.transform(arr1), scaler)
                                
(input, output) = split_time_series(5, load_data())
merged = merge_time_series(input, output)

print(input)
print(output)
print(merged)

[[[4366.64 4411.01 4287.11 4356.45]
  [4356.32 4417.35 4222.62 4410.13]
  [4471.38 4494.52 4395.34 4397.94]
  [4547.35 4602.11 4477.95 4482.73]
  [4588.03 4611.55 4530.2  4532.76]]

 [[4356.32 4417.35 4222.62 4410.13]
  [4471.38 4494.52 4395.34 4397.94]
  [4547.35 4602.11 4477.95 4482.73]
  [4588.03 4611.55 4530.2  4532.76]
  [4632.24 4632.24 4568.7  4577.11]]

 [[4471.38 4494.52 4395.34 4397.94]
  [4547.35 4602.11 4477.95 4482.73]
  [4588.03 4611.55 4530.2  4532.76]
  [4632.24 4632.24 4568.7  4577.11]
  [4637.99 4665.13 4614.75 4662.85]]

 ...

 [[2457.77 2571.42 2407.53 2475.56]
  [2344.44 2449.71 2344.44 2447.33]
  [2290.71 2300.73 2191.86 2237.4 ]
  [2431.94 2453.01 2295.56 2304.92]
  [2393.48 2466.97 2319.78 2409.39]]

 [[2344.44 2449.71 2344.44 2447.33]
  [2290.71 2300.73 2191.86 2237.4 ]
  [2431.94 2453.01 2295.56 2304.92]
  [2393.48 2466.97 2319.78 2409.39]
  [2436.5  2453.57 2280.52 2398.1 ]]

 [[2290.71 2300.73 2191.86 2237.4 ]
  [2431.94 2453.01 2295.56 2304.92]
  [2393.48 2

# Creating GAN

In [76]:
class LSTMGAN():
    def __init__(self, t, f, data):
        self.data = data
        self.time_series_len = t
        self.feature_len = f
        self.gen_shape = (self.time_series_len, self.feature_len)
        self.dis_shape = (self.time_series_len+1, self.feature_len)

        optimizer = Adam(0.0001, 0.4)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        # Trains the generator to imitate input data
        self.generator.compile(loss='mean_squared_error',
            optimizer=optimizer)
        
        # The generator takes noise as input and generates song
        noise = Input(shape=self.gen_shape)
        gen_output = self.generator(noise)
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(merge_time_series(noise, gen_output))

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(noise, valid)
        
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        

    def build_generator(self):

        model = Sequential()
        model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=self.gen_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(128)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(4))
        # model.summary()

        noise = Input(shape=self.gen_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        
        model.add(Bidirectional(LSTM(128, activation = 'relu', return_sequences=True), input_shape=self.dis_shape))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(128, activation = 'relu')))
        model.add(Dense(1, activation = 'linear'))
        #model.summary()

        img = Input(shape=self.dis_shape)
        validity = model(img)

        return Model(img, validity)
    
    
    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train_input, X_train_output) = split_time_series(self.time_series_len, self.data)

        # normalize

        # Adversarial ground truths
        valid = np.ones(batch_size)
        fake = np.zeros(batch_size)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # -------------------)--
            # Select a random half of songs
            idx = np.random.randint(0, X_train_input.shape[0], batch_size)
            real_input= X_train_input[idx]
            real_output= X_train_output[idx]
            
            #standardize data
            def standardize_fn(input_t, output_t) : 
                scaler = StandardScaler();
                scaler.fit(input_t)
                return (scaler.transform(input_t), scaler.transform([output_t])[0], scaler)
            
            zipped = [ standardize_fn(i, o) for (i, o) in zip(real_input, real_output)]
            (real_input, real_output, scalers) = zip(*zipped)
            
            # Sample noise and generate a batch of new prices
            noise = np.random.normal(0, 1, (batch_size,self.time_series_len,self.feature_len))
            gen_output = self.generator.predict(noise, batch_size = batch_size)
            real_series = merge_time_series(real_input,real_output)
            fake_series = merge_time_series(real_input,gen_output)
            print("input:\n", scalers[0].inverse_transform( real_input[0]))
            print("fake_output:\n", scalers[0].inverse_transform( [gen_output[0]]))
            print("real_output:\n", scalers[0].inverse_transform( [real_output[0]]))

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(real_series, valid)
            d_loss_fake = self.discriminator.train_on_batch(fake_series, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------
            # First training (wants generator to imitate real data)
            #g_loss1 = self.generator.train_on_batch( np.array(real_input), np.array(real_output))
            
            # Second training (wants discriminator to mistake songs as real)
            #g_loss2 = self.combined.train_on_batch(noise, valid)
            #g_loss = 0.5 * np.add(g_loss1, g_loss2)
            g_loss = self.combined.train_on_batch(noise, valid)



            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save model
            if epoch % save_interval == 0:
                self.generator.save("LSTM_generator.h5")

# Model Summary
I couldn't train the model on this online notebook so I trained it locally for 1000 epochs and uploaded the h5 file.

In [77]:
lstmgan = LSTMGAN(5, 4, load_data())
lstmgan.train(epochs=1000, batch_size=50, save_interval=100)

input:
 [[2578.28 2676.85 2574.57 2663.68]
 [2514.92 2538.18 2459.96 2488.65]
 [2458.54 2533.22 2455.79 2526.9 ]
 [2498.08 2522.75 2447.49 2470.5 ]
 [2614.69 2641.39 2571.15 2584.59]]
fake_output:
 [[2532.80157862 2582.31648315 2503.2647609  2546.65819059]]
real_output:
 [[2558.98 2631.8  2545.28 2626.65]]
0 [D loss: 5.193274, acc.: 50.00%] [G loss: 7.019440]
input:
 [[4477.09 4485.87 4443.8  4473.75]
 [4447.49 4486.87 4438.37 4480.7 ]
 [4479.33 4485.68 4435.46 4443.05]
 [4474.81 4492.99 4445.7  4468.73]
 [4506.92 4520.47 4457.66 4458.58]]
fake_output:
 [[4477.16599024 4494.03553023 4444.28321356 4464.86181748]]
real_output:
 [[4513.02 4529.9  4492.07 4493.28]]
1 [D loss: 4.099460, acc.: 50.00%] [G loss: 6.206120]
input:
 [[3371.88 3379.97 3329.27 3331.84]
 [3453.6  3479.15 3349.63 3426.96]
 [3564.74 3564.85 3427.41 3455.06]
 [3543.76 3588.11 3535.23 3580.84]
 [3507.44 3528.03 3494.6  3526.65]]
fake_output:
 [[3488.74738153 3507.92944082 3428.54601833 3464.67041259]]
real_output:
 [[35

22 [D loss: 1.110656, acc.: 50.00%] [G loss: 2.136855]
input:
 [[3566.82 3589.81 3552.77 3577.59]
 [3579.31 3581.23 3556.85 3557.54]
 [3559.41 3585.22 3543.84 3581.87]
 [3612.09 3619.09 3567.33 3567.79]
 [3610.31 3623.11 3588.68 3609.53]]
fake_output:
 [[3585.71576386 3599.37937721 3564.16196517 3579.28270469]]
real_output:
 [[3600.16 3628.51 3600.16 3626.91]]
23 [D loss: 1.078706, acc.: 50.00%] [G loss: 2.089335]
input:
 [[4415.95 4416.17 4400.23 4402.66]
 [4392.74 4423.79 4373.   4423.15]
 [4406.86 4422.18 4384.81 4387.16]
 [4395.12 4412.25 4389.65 4395.26]
 [4403.59 4429.97 4403.59 4419.15]]
fake_output:
 [[4402.9459984  4420.59499252 4391.66604673 4405.91812363]]
real_output:
 [[4402.95 4415.47 4387.01 4400.64]]
24 [D loss: 1.075640, acc.: 50.00%] [G loss: 2.054240]
input:
 [[2829.95 2865.01 2816.78 2863.7 ]
 [2794.54 2852.8  2766.64 2852.5 ]
 [2865.86 2874.14 2793.15 2820.  ]
 [2939.5  2945.82 2869.59 2870.12]
 [2915.46 2944.25 2903.44 2930.32]]
fake_output:
 [[2870.45121788 2895.

input:
 [[3552.57 3593.66 3552.57 3585.15]
 [3562.67 3569.02 3518.58 3537.01]
 [3563.22 3581.16 3557.   3572.66]
 [3543.26 3557.22 3511.91 3545.53]
 [3583.04 3645.99 3547.48 3550.5 ]]
fake_output:
 [[3560.57647683 3586.53004718 3543.42917206 3560.00838806]]
real_output:
 [[3508.34 3521.58 3484.34 3509.44]]
46 [D loss: 0.874573, acc.: 50.67%] [G loss: 1.656256]
input:
 [[3350.92 3357.92 3327.54 3335.47]
 [3333.9  3360.74 3332.91 3351.6 ]
 [3236.66 3306.88 3228.44 3298.46]
 [3226.14 3278.7  3209.45 3246.59]
 [3320.11 3323.35 3232.57 3236.92]]
fake_output:
 [[3294.34329202 3323.14752324 3284.84002207 3299.38062435]]
real_output:
 [[3295.75 3320.31 3270.95 3315.57]]
47 [D loss: 0.868421, acc.: 50.50%] [G loss: 1.673684]
input:
 [[4578.69 4598.53 4569.17 4574.79]
 [4553.69 4572.62 4537.36 4566.48]
 [4546.12 4559.67 4524.   4544.9 ]
 [4532.24 4551.44 4526.89 4549.78]
 [4524.42 4540.87 4524.4  4536.19]]
fake_output:
 [[4547.29975739 4562.98239873 4541.77566355 4555.79402164]]
real_output:
 [[

69 [D loss: 0.768850, acc.: 52.00%] [G loss: 1.350172]
input:
 [[2939.5  2945.82 2869.59 2870.12]
 [2915.46 2944.25 2903.44 2930.32]
 [2908.83 2932.16 2902.88 2929.8 ]
 [2878.26 2901.92 2876.48 2881.19]
 [2883.14 2891.11 2847.65 2848.42]]
fake_output:
 [[2905.65487398 2909.72227182 2916.34556853 2916.18679339]]
real_output:
 [[2868.88 2898.23 2863.55 2868.44]]
70 [D loss: 0.787232, acc.: 51.50%] [G loss: 1.328577]
input:
 [[4368.31 4374.89 4342.09 4350.65]
 [4385.44 4415.88 4360.59 4361.19]
 [4406.51 4412.02 4386.22 4391.34]
 [4383.73 4429.97 4383.73 4399.76]
 [4319.57 4365.57 4290.49 4363.55]]
fake_output:
 [[4373.99601064 4367.85692504 4468.92799084 4404.29698709]]
real_output:
 [[4309.87 4369.23 4309.87 4345.72]]
71 [D loss: 0.756128, acc.: 52.17%] [G loss: 1.304804]
input:
 [[3131.   3153.45 3076.06 3124.74]
 [2993.76 3079.76 2965.66 3066.59]
 [3071.04 3088.42 2984.47 3041.31]
 [3123.53 3123.53 2999.49 3002.1 ]
 [3213.42 3223.27 3181.49 3190.14]]
fake_output:
 [[3108.72855211 3082.

input:
 [[4403.59 4429.97 4403.59 4419.15]
 [4402.95 4415.47 4387.01 4400.64]
 [4416.38 4416.38 4372.51 4401.46]
 [4409.58 4422.73 4405.45 4422.3 ]
 [4381.2  4415.18 4381.2  4411.79]]
fake_output:
 [[4396.5569941  4409.68742297 4513.76962097 4459.53672742]]
real_output:
 [[4361.27 4369.87 4350.06 4367.48]]
93 [D loss: 0.926094, acc.: 41.67%] [G loss: 0.882883]
input:
 [[4096.11 4129.48 4095.51 4128.8 ]
 [4089.95 4098.19 4082.54 4097.17]
 [4074.29 4083.13 4068.31 4079.95]
 [4075.57 4086.23 4068.14 4073.94]
 [4034.44 4083.42 4034.44 4077.91]]
fake_output:
 [[4059.18393615 4062.64352008 4296.49309301 4219.83442304]]
real_output:
 [[3992.78 4020.63 3992.78 4019.87]]
94 [D loss: 0.887904, acc.: 41.00%] [G loss: 0.879366]
input:
 [[3508.34 3521.58 3484.34 3509.44]
 [3485.74 3529.05 3485.74 3510.45]
 [3406.46 3486.25 3405.17 3443.44]
 [3336.25 3389.49 3336.25 3369.16]
 [3296.2  3330.14 3279.74 3310.24]]
fake_output:
 [[3341.38307066 3301.13524743 4405.11848311 3985.3443475 ]]
real_output:
 [[

...

input:
 [[2948.05 2956.76 2933.59 2955.45]
 [2969.95 2978.5  2938.57 2948.51]
 [2953.63 2980.29 2953.63 2971.61]
 [2948.59 2964.21 2922.35 2922.94]
 [2913.86 2968.09 2913.86 2953.91]]
fake_output:
 [[2994.06362151 2918.61129021 2924.52492806 2878.29187183]]
real_output:
 [[2829.95 2865.01 2816.78 2863.7 ]]
904 [D loss: 0.715709, acc.: 53.83%] [G loss: 0.954208]
input:
 [[4188.25 4193.35 4176.22 4186.72]
 [4185.03 4194.19 4182.36 4187.62]
 [4138.78 4194.17 4138.78 4180.17]
 [4170.46 4179.57 4123.69 4134.98]
 [4128.42 4175.02 4126.35 4173.42]]
fake_output:
 [[4191.31502193 4178.51178546 4163.56444163 4164.32137971]]
real_output:
 [[4159.18 4159.18 4118.38 4134.94]]
905 [D loss: 0.755926, acc.: 46.67%] [G loss: 0.944751]
input:
 [[3515.47 3527.94 3480.55 3488.67]
 [3534.01 3534.01 3500.86 3511.93]
 [3500.02 3549.85 3499.61 3534.22]
 [3459.67 3482.34 3458.07 3477.13]
 [3434.28 3447.28 3428.15 3446.83]]
fake_output:
 [[3548.30416007 3443.20107069 3486.53449595 3469.27329789]]
real_output:
 

927 [D loss: 0.925152, acc.: 44.67%] [G loss: 1.112775]
input:
 [[3213.32 3222.71 3193.11 3207.18]
 [3199.92 3233.13 3196.   3232.39]
 [3163.84 3211.72 3163.84 3193.93]
 [3111.56 3128.91 3090.41 3112.35]
 [3098.9  3130.94 3098.9  3122.87]]
fake_output:
 [[3432.38142135 3194.21456092 3412.03302765 3322.01661203]]
real_output:
 [[3064.78 3081.07 3051.64 3080.82]]
928 [D loss: 0.905032, acc.: 42.17%] [G loss: 1.048354]
input:
 [[2776.99 2818.57 2762.36 2789.82]
 [2685.   2760.75 2663.3  2749.98]
 [2738.65 2756.89 2657.67 2659.41]
 [2578.28 2676.85 2574.57 2663.68]
 [2514.92 2538.18 2459.96 2488.65]]
fake_output:
 [[2813.75639268 2782.15263931 2820.99793716 2832.24821977]]
real_output:
 [[2458.54 2533.22 2455.79 2526.9 ]]
929 [D loss: 0.884414, acc.: 43.33%] [G loss: 1.168148]
input:
 [[3367.27 3409.57 3367.27 3408.63]
 [3338.94 3369.1  3323.69 3348.44]
 [3385.87 3397.18 3361.39 3380.8 ]
 [3341.21 3393.56 3340.47 3363.  ]
 [3350.92 3357.92 3327.54 3335.47]]
fake_output:
 [[3471.5312927  33

951 [D loss: 0.668026, acc.: 62.83%] [G loss: 3.382694]
input:
 [[3296.2  3330.14 3279.74 3310.24]
 [3293.59 3304.93 3233.94 3269.96]
 [3277.17 3341.05 3259.82 3310.11]
 [3342.48 3342.48 3268.89 3271.03]
 [3403.15 3409.51 3388.71 3390.68]]
fake_output:
 [[3421.04582404 3444.27051496 3465.85124139 3494.35217964]]
real_output:
 [[3441.42 3441.42 3364.86 3400.97]]
952 [D loss: 0.634076, acc.: 65.33%] [G loss: 2.558321]
input:
 [[3234.27 3243.72 3216.17 3218.44]
 [3219.84 3241.43 3214.25 3239.41]
 [3218.58 3227.26 3200.05 3215.63]
 [3271.64 3279.99 3222.66 3235.66]
 [3254.86 3279.32 3253.1  3276.02]]
fake_output:
 [[3363.4942249  3330.67176986 3315.11850788 3388.77894339]]
real_output:
 [[3268.52 3277.29 3247.77 3257.3 ]]
953 [D loss: 0.704980, acc.: 58.83%] [G loss: 2.835006]
input:
 [[4174.14 4191.31 4170.75 4185.47]
 [4139.76 4173.49 4139.76 4170.42]
 [4141.58 4151.69 4120.87 4124.66]
 [4130.1  4148.   4124.43 4141.59]
 [4124.71 4131.76 4114.82 4127.99]]
fake_output:
 [[4220.67131893 42

975 [D loss: 0.565144, acc.: 71.17%] [G loss: 2.086520]
input:
 [[3094.42 3120.92 3079.39 3117.86]
 [3140.29 3155.53 3083.11 3097.74]
 [3101.64 3120.   3093.51 3115.34]
 [3136.13 3141.16 3108.03 3113.49]
 [3131.   3153.45 3076.06 3124.74]]
fake_output:
 [[3216.00974288 3196.95215423 3104.19848966 3137.00007205]]
real_output:
 [[2993.76 3079.76 2965.66 3066.59]]
976 [D loss: 0.531050, acc.: 71.67%] [G loss: 2.017921]
input:
 [[3755.75 3830.5  3755.75 3787.38]
 [3836.83 3836.83 3732.48 3750.77]
 [3862.96 3870.9  3847.78 3849.62]
 [3851.68 3859.23 3797.16 3855.36]
 [3844.24 3852.31 3830.41 3841.47]]
fake_output:
 [[3866.75405137 3880.79490933 3866.90341323 3904.76518217]]
real_output:
 [[3857.46 3861.45 3845.05 3853.07]]
977 [D loss: 0.603210, acc.: 68.50%] [G loss: 2.148510]
input:
 [[3213.32 3222.71 3193.11 3207.18]
 [3199.92 3233.13 3196.   3232.39]
 [3163.84 3211.72 3163.84 3193.93]
 [3111.56 3128.91 3090.41 3112.35]
 [3098.9  3130.94 3098.9  3122.87]]
fake_output:
 [[3391.91273431 33

input:
 [[4642.99 4660.47 4606.52 4634.09]
 [4710.3  4710.3  4667.6  4668.97]
 [4687.64 4713.57 4670.24 4712.02]
 [4691.   4695.26 4665.98 4667.45]
 [4690.86 4705.06 4674.52 4701.21]]
fake_output:
 [[4779.9000526  4747.86248264 4678.04875827 4698.00747493]]
real_output:
 [[4631.97 4694.04 4631.97 4686.75]]
999 [D loss: 0.534245, acc.: 71.00%] [G loss: 1.731105]


Loading pretrained model

Installinging Mido Library

# Generating Melody
Generating random input and letting model predict output

In [143]:

random = np.random.normal(0,1,(1,5,4))

predict = lstmgan.generator(random)

print(predict)

tf.Tensor([[ 0.08106427 -6.513324   -8.422126   -9.677593  ]], shape=(1, 4), dtype=float32)


# Back to MIDI
Save generated melody back to a .mid file

In [ ]:
midler = MidiFile()
track = MidiTrack()
midler.tracks.append(track)
track.append(Message('program_change', program=2, time=0))
for x in range(16):
    track.append(Message('note_on', note=int(predict[0][x][0]), velocity=64, time=20))
    track.append(Message('note_off', note=int(predict[0][x][0]), velocity=64, time=20))
    midler.save('new_song.mid')